<a href="https://colab.research.google.com/github/EjbejaranosAI/EmotionUnify/blob/main/Audio_Meld_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and install libraries and dataset

In [2]:
!pip install transformers lazypredict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00


### Module to set the dataset

In [3]:
"""
This module is to download multimodal datasets and store them in the dataset directory
"""
import os
import os.path
from os import path
import logging
import shutil
import subprocess
import tarfile


def download_meld_dataset():
    """
    Download MELD dataset and store it into the dataset directory

    """
    if path.exists("/content/dataset")== False:
      os.mkdir("/content/datasets")

    os.chdir('/content/datasets')
    try:
        # Add logging configuration
        logging.basicConfig(filename='download.log', level=logging.INFO)
        logging.info('Downloading MELD dataset')

        # Use wget to download the dataset
        try:
            subprocess.run(['wget', 'https://huggingface.co/datasets/declare-lab/MELD/resolve/main/MELD.Raw.tar.gz'])
        except:
            subprocess.run(['wget','http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz'])

        # Extract the downloaded dataset
        subprocess.run(['tar', '-xvf', 'MELD.Raw.tar.gz'])

        # Remove the downloaded tar.gz file
        os.remove('MELD.Raw.tar.gz')

        # Rename the extracted directory to "MELD"
        os.rename('MELD.Raw', 'MELD')

        # Extract the inner tar files
        files = os.listdir("./MELD")
        files_to_uncompress = []
        for file in files:
            if file.endswith('.tar.gz'):
                print("tar.gz file found")
                files_to_uncompress.append(file)
        if len(files_to_uncompress) >0:
            for file in files_to_uncompress:
                # I want to extract the files in the same directory that was found it and later deleter the tar.gz files
                tar = tarfile.open("./MELD/"+file)
                tar.extractall("./MELD/")
                tar.close()
                os.remove("./MELD/"+file)

        else:
            print("No tar.gz files found")
    except:
        logging.error('Error in downloading MELD dataset')
        print('Error in downloading MELD dataset')


if __name__ == "__main__":
    download_meld_dataset()

tar.gz file found
tar.gz file found
tar.gz file found


# Reading dataset

In [3]:
import os
import numpy as np
import pandas as pd

path_dataset = "/content/datasets/MELD"

# In[2]:
def load_data(path):
    csv_file_data = []
    files_to_uncompress = []
    files = os.listdir(path)
    print(files)
    # get the csv files from the files and read it
    for file in files:
        if file.endswith(".csv"):
            print(file)
            csv_file_data.append({file:pd.read_csv(path + '/' + file)})


    print(len(csv_file_data))



    return files, csv_file_data

files , csv_file_data = load_data(path_dataset)

['dev_sent_emo.csv', 'test_sent_emo.csv', 'dev_splits_complete', '._train_splits', 'output_repeated_splits_test', 'train_sent_emo.csv', 'train_splits', '._output_repeated_splits_test', 'README.txt']
dev_sent_emo.csv
test_sent_emo.csv
train_sent_emo.csv
3


In [4]:
train_df = csv_file_data[2]['train_sent_emo.csv']
dev_df = csv_file_data[0]['dev_sent_emo.csv']
test_df = csv_file_data[1]['test_sent_emo.csv']

In [5]:
df_train = train_df
df_dev = dev_df
df_test = test_df
print(len(df_train))
df_train.head()

9989


,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [6]:
# Define mapping dictionaries
emotion_mapping = {
    'neutral': [1, 0, 0, 0, 0, 0, 0],
    'surprise': [0, 1, 0, 0, 0, 0, 0],
    'fear': [0, 0, 1, 0, 0, 0, 0],
    'sadness': [0, 0, 0, 1, 0, 0, 0],
    'joy': [0, 0, 0, 0, 1, 0, 0],
    'disgust': [0, 0, 0, 0, 0, 1, 0],
    'anger': [0, 0, 0, 0, 0, 0, 1]
}

sentiment_mapping = {
    'neutral': 0,
    'positive': 1,
    'negative': 2
}

In [7]:
# Process the training, development, and test datasets
df_train['video_id'] = df_train.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_dev['video_id'] = df_dev.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_test['video_id'] = df_test.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)

df_train['Emotion_encoded'] = df_train['Emotion'].map(emotion_mapping)
df_train['Sentiment_encoded'] = df_train['Sentiment'].map(sentiment_mapping)
df_processed_train = df_train[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_dev['Emotion_encoded'] = df_dev['Emotion'].map(emotion_mapping)
df_dev['Sentiment_encoded'] = df_dev['Sentiment'].map(sentiment_mapping)
df_processed_dev = df_dev[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_test['Emotion_encoded'] = df_test['Emotion'].map(emotion_mapping)
df_test['Sentiment_encoded'] = df_test['Sentiment'].map(sentiment_mapping)
df_processed_test = df_test[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

In [8]:
df_processed_train

,video_id,Utterance,Emotion_encoded,Sentiment_encoded
0,dia0_utt0.mp4,also I was the point person on my companys tr...,"[1, 0, 0, 0, 0, 0, 0]",0
1,dia0_utt1.mp4,You mustve had your hands full.,"[1, 0, 0, 0, 0, 0, 0]",0
2,dia0_utt2.mp4,That I did. That I did.,"[1, 0, 0, 0, 0, 0, 0]",0
3,dia0_utt3.mp4,So lets talk a little bit about your duties.,"[1, 0, 0, 0, 0, 0, 0]",0
4,dia0_utt4.mp4,My duties? All right.,"[0, 1, 0, 0, 0, 0, 0]",1
...,...,...,...,...
9984,dia1038_utt13.mp4,You or me?,"[1, 0, 0, 0, 0, 0, 0]",0
9985,dia1038_utt14.mp4,"I got it. Uh, Joey, women don't have Adam's ap...","[1, 0, 0, 0, 0, 0, 0]",0
9986,dia1038_utt15.mp4,"You guys are messing with me, right?","[0, 1, 0, 0, 0, 0, 0]",1
9987,dia1038_utt16.mp4,Yeah.,"[1, 0, 0, 0, 0, 0, 0]",0


In [9]:
print(f"Number of videos in train: {df_processed_train.shape}")
print(f"Number of videos in dev: {df_processed_dev.shape}")
print(f"Number of videos in test: {df_processed_test.shape}")

Number of videos in train: (9989, 4)
Number of videos in dev: (1109, 4)
Number of videos in test: (2610, 4)


# Feature extraction in text

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bars

# Define a function to extract BERT features
def extract_bert_features(df):
    # Tokenize utterances and obtain BERT features with padding
    max_length = 128  # Define your desired maximum sequence length

    # Tokenize utterances and obtain BERT features
    utterances = df['Utterance'].tolist()
    input_ids = []
    attention_masks = []

    # Use tqdm to add progress bars
    for utterance in tqdm(utterances, desc="Tokenizing Utterances"):
        tokens = tokenizer.encode(utterance, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length')
        input_ids.append(tokens)
        attention_masks.append([1] * len(tokens))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    with torch.no_grad():
        bert_outputs = model(input_ids, attention_mask=attention_masks)

    # Use BERT features to train a logistic regression classifier
    X = bert_outputs[0][:, 0, :].numpy()  # Using the [CLS] token's representation
    y = df['Sentiment_encoded'].tolist()

    return X, y

In [ ]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


# Extract BERT features for the training, development, and test datasets
X_train, y_train = extract_bert_features(df_processed_train)
np.save('train_text_features.npy', X_train)

X_dev, y_dev = extract_bert_features(df_processed_dev)
np.save('dev_text_features.npy', X_dev)

X_test, y_test = extract_bert_features(df_processed_test)
np.save('test_text_features.npy', X_test)

Tokenizing Utterances: 100%|██████████| 2610/2610 [00:00<00:00, 3851.59it/s]


### Text classifier

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert one-hot encoded labels to single-class labels
#y_train_single = np.argmax(y_train, axis=1)
#y_test_single = np.argmax(y_test, axis=1)


classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6754789272030651


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|█████████▋| 28/29 [12:34<00:35, 35.52s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040925 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195839
[LightGBM] [Info] Number of data points in the train set: 9989, number of used features: 768
[LightGBM] [Info] Start training from score -0.751797
[LightGBM] [Info] Start training from score -1.453901
[LightGBM] [Info] Start training from score -1.221376


100%|██████████| 29/29 [12:45<00:00, 26.41s/it]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LogisticRegression                 0.67               0.63    None      0.67   
LinearDiscriminantAnalysis         0.66               0.62    None      0.65   
RidgeClassifierCV                  0.67               0.62    None      0.66   
RidgeClassifier                    0.66               0.61    None      0.65   
LinearSVC                          0.64               0.60    None      0.63   
CalibratedClassifierCV             0.65               0.59    None      0.64   
SGDClassifier                      0.61               0.58    None      0.60   
XGBClassifier                      0.64               0.57    None      0.62   
SVC                                0.65               0.57    None      0.62   
NuSVC                              0.65               0.57    None      0.63   
QuadraticDiscriminantAnalysis      0.65 

In [ ]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LogisticRegression,0.67,0.63,None,0.67,1.84
LinearDiscriminantAnalysis,0.66,0.62,None,0.65,2.19
RidgeClassifierCV,0.67,0.62,None,0.66,1.57
RidgeClassifier,0.66,0.61,None,0.65,0.70
LinearSVC,0.64,0.60,None,0.63,65.27
CalibratedClassifierCV,0.65,0.59,None,0.64,261.63
SGDClassifier,0.61,0.58,None,0.60,7.53
XGBClassifier,0.64,0.57,None,0.62,104.50
SVC,0.65,0.57,None,0.62,51.76


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Accuracy'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Balanced Accuracy'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['F1 Score'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Time Taken'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(models, *[[['Accuracy', 'Balanced Accuracy'], ['Balanced Accuracy', 'F1 Score'], ['F1 Score', 'Time Taken']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'Accuracy', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'Balanced Accuracy', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'F1 Score', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(models, *['Time Taken', 'count()', None], **{})
chart

In [ ]:
import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(models, *['Accuracy'], **{})
chart

# Feature extraction in audio

In [10]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import torchaudio

# Check for available GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x


class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits


def process_func(
    wavs,
    sampling_rate: int
    # embeddings: bool = False,
):
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    # wavs = pad_sequence(wavs, batch_first=True)
    # load model from hub
    device = 'cpu'
    model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = EmotionModel.from_pretrained(model_name).to(device)
    y = y.to(device)  # Move tensor to GPU

    y = processor([wav.cpu().numpy() for wav in wavs],
                   sampling_rate=sampling_rate,
                   return_tensors="pt",
                   padding="longest"
        )
    y = y['input_values']
    y = y.to(device)


    y = model(y)

    return {
        'hidden_states': y[0].cpu(),
        'logits': y[1].cpu()
    }


In [11]:
!pip install -q pydub

### Convert from mp4 to wav

In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from pydub import AudioSegment
import os
import pandas as pd
from tqdm import tqdm

def convert_video_to_wav(filename, input_folder, output_folder):
    mp4_path = os.path.join(input_folder, filename)
    wav_path = os.path.join(output_folder, filename.replace(".mp4", ".wav"))
    try:
        audio = AudioSegment.from_file(mp4_path, format="mp4")
        audio.export(wav_path, format="wav")
        return f"Converted {filename} to {wav_path}."
    except Exception as e:
        return f"Could not convert {filename} due to {e}. Skipping."

def executor_convert_video_to_audio(df, input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    not_processed = []
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(convert_video_to_wav, row['video_id'], input_folder, output_folder): row for index, row in df.iterrows()}
        for future in tqdm(as_completed(futures), total=len(futures)):
            row = futures[future]
            try:
                print(future.result())
            except Exception as e:
                not_processed.append(row['video_id'])

    num_videos = df.shape[0]
    num_audios = len(os.listdir(output_folder))
    print(f"Number of videos: {num_videos}")
    print(f"Number of audios: {num_audios}")
    print(f"Number of audios not processed: {len(not_processed)}")
    print(f"Audios not processed: {not_processed}")


# Train
video_directory = "/content/datasets/MELD/train_splits"
output_directory = "/content/datasets/MELD/train_splits_wav"
executor_convert_video_to_audio(df_processed_train, video_directory, output_directory)

# Dev
video_directory = "/content/datasets/MELD/dev_splits_complete"
output_directory = "/content/datasets/MELD/dev_splits_complete_wav"
executor_convert_video_to_audio(df_processed_dev, video_directory, output_directory)

# Test
video_directory = "/content/datasets/MELD/output_repeated_splits_test"
output_directory = "/content/datasets/MELD/output_repeated_splits_test_wav"
executor_convert_video_to_audio(df_processed_test, video_directory, output_directory)


  0%|          | 34/9989 [00:00<01:22, 120.45it/s]

Converted dia1_utt5.mp4 to /content/datasets/MELD/train_splits_wav/dia1_utt5.wav.
Converted dia0_utt3.mp4 to /content/datasets/MELD/train_splits_wav/dia0_utt3.wav.
Converted dia0_utt9.mp4 to /content/datasets/MELD/train_splits_wav/dia0_utt9.wav.
Converted dia2_utt2.mp4 to /content/datasets/MELD/train_splits_wav/dia2_utt2.wav.
Converted dia2_utt6.mp4 to /content/datasets/MELD/train_splits_wav/dia2_utt6.wav.
Converted dia0_utt4.mp4 to /content/datasets/MELD/train_splits_wav/dia0_utt4.wav.
Converted dia1_utt0.mp4 to /content/datasets/MELD/train_splits_wav/dia1_utt0.wav.
Converted dia2_utt10.mp4 to /content/datasets/MELD/train_splits_wav/dia2_utt10.wav.
Converted dia1_utt4.mp4 to /content/datasets/MELD/train_splits_wav/dia1_utt4.wav.
Converted dia0_utt6.mp4 to /content/datasets/MELD/train_splits_wav/dia0_utt6.wav.
Converted dia0_utt5.mp4 to /content/datasets/MELD/train_splits_wav/dia0_utt5.wav.
Converted dia2_utt1.mp4 to /content/datasets/MELD/train_splits_wav/dia2_utt1.wav.
Converted dia0

  0%|          | 46/9989 [00:01<06:02, 27.41it/s] 

Converted dia4_utt3.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt3.wav.
Converted dia4_utt2.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt2.wav.
Converted dia4_utt6.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt6.wav.
Converted dia4_utt5.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt5.wav.
Converted dia4_utt4.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt4.wav.
Converted dia4_utt7.mp4 to /content/datasets/MELD/train_splits_wav/dia4_utt7.wav.


  1%|          | 53/9989 [00:01<07:27, 22.23it/s]

...

Converted dia279_utt7.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt7.wav.
Converted dia279_utt9.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt9.wav.


100%|█████████▉| 2608/2610 [04:39<00:00, 13.64it/s]

Converted dia279_utt13.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt13.wav.
Converted dia279_utt10.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt10.wav.
Converted dia279_utt8.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt8.wav.
Converted dia279_utt15.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt15.wav.
Converted dia279_utt14.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt14.wav.
Converted dia279_utt11.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt11.wav.


100%|██████████| 2610/2610 [04:39<00:00,  9.35it/s]

Converted dia279_utt12.mp4 to /content/datasets/MELD/output_repeated_splits_test_wav/dia279_utt12.wav.
Number of videos: 2610
Number of audios: 2610
Number of audios not processed: 0
Audios not processed: []


In [13]:
def checking_files(path_video, path_audio):
  video_files = os.listdir(path_video)
  audio_files = os.listdir(path_audio)

  print(f"Video files:{len(video_files)} audio files:{len(audio_files)}")

# Train
video_directory = "/content/datasets/MELD/train_splits"
output_directory = "/content/datasets/MELD/train_splits_wav"
checking_files(video_directory, output_directory)

# Dev
video_directory = "/content/datasets/MELD/dev_splits_complete"
output_directory = "/content/datasets/MELD/dev_splits_complete_wav"
checking_files(video_directory, output_directory)

# Test
video_directory = "/content/datasets/MELD/output_repeated_splits_test"
output_directory = "/content/datasets/MELD/output_repeated_splits_test_wav"
checking_files(video_directory, output_directory)

Video files:9989 audio files:9988
Video files:1112 audio files:1108
Video files:4807 audio files:2610


### Extracting features in all the dataset

In [1]:
from torchaudio.transforms import Resample
from torch.nn import functional as nnf
from tqdm import tqdm
import soundfile as sf

def get_feature_extraction_from_audios(path_audio,name_file_output):
    # Initialize an empty dictionary to store features
    feature_dict = {}

    # Your directory containing all .wav files
    audio_directory = path_audio

    # Initialize resampler
    resampler = Resample(orig_freq=48000, new_freq=16000)

    # Iterate through each .wav file
    for audio_file in tqdm(os.listdir(audio_directory)):
        if audio_file.endswith('.wav'):
            # Read the audio file
            audio_path = os.path.join(audio_directory, audio_file)
            audio_data, sampling_rate = sf.read(audio_path)

            # Convert to mono if stereo
            if audio_data.ndim > 1:
                audio_data = audio_data.mean(axis=1)

            # Resample audio
            audio_data_resampled = resampler(torch.tensor(audio_data).float())

            # Pad audio if too short
            min_length = 16000  # 1 second at 16kHz
            if audio_data_resampled.shape[0] < min_length:
                padding_needed = min_length - audio_data_resampled.shape[0]
                audio_data_resampled = nnf.pad(audio_data_resampled, (0, padding_needed))

            # Extract features using your function
            features = process_func(audio_data_resampled.unsqueeze(0), 16000)  # Now using 16000 Hz

            # Store features
            feature_dict[audio_file] = features['hidden_states'].detach().cpu().numpy()

    # Save features to a .npy file
    np.save(name_file_output, feature_dict)

if __name__ == "__main__":
    get_feature_extraction_from_audios("/content/datasets/MELD/train_splits_wav",'audio__train_features.npy')
    get_feature_extraction_from_audios("/content/datasets/MELD/output_repeated_splits_test_wav",'audio__test_features.npy')
    get_feature_extraction_from_audios("/content/datasets/MELD/dev_splits_complete_wav",'audio__dev_features.npy')

NameError: ignored

# Video extraction

In [ ]:
!pip install opencv-python
!pip install torch torchvision
!pip install transformers
!pip install timm

In [ ]:
import cv2
import torch
import torchvision.models as models
import timm  # for Vision Transformer
from transformers import BertModel
import os
import numpy as np
from tqdm import tqdm

def extract_frames(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def cnn_feature_extraction(model, model_name, frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to fit most CNN input
    tensor = torch.tensor(frame).float().unsqueeze(0).permute(0, 3, 1, 2) / 255.0
    with torch.no_grad():
        features = model(tensor)
    return features.squeeze().numpy()

def process_video(video_path, models):
    frames = extract_frames(video_path)

    combined_features = []
    for model_name, model in models.items():
        frame_features = [cnn_feature_extraction(model, model_name, frame) for frame in frames]
        combined_features.append(np.stack(frame_features))

    combined_features = np.concatenate(combined_features, axis=-1)  # Stacking features

    # Save to disk for future use
    np.save(f"{video_path}_features.npy", combined_features)

def main():
    # Initialize models with explicit namespaces to avoid name conflicts
    model_dict = {
        'resnet': models.resnet18(weights=ResNet18_Weights.ImageNet1K_V1),
        'vgg': models.vgg16(weights=VGG16_Weights.ImageNet1K_V1),
        'vit': timm.create_model("vit_base_patch16_224", pretrained=True)
    }

    # Remove classification heads
    for key in model_dict:
        if key != 'vit':  # Vision Transformer doesn't have a straightforward "head" like other CNNs
            model_dict[key] = torch.nn.Sequential(*(list(model_dict[key].children())[:-1]))

    video_folder = "./videos"  # Replace with your folder path
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

    for video_file in tqdm(video_files):
        video_path = os.path.join(video_folder, video_file)
        process_video(video_path, model_dict)

if __name__ == "__main__":
    main()


ModuleNotFoundError: ignored

# Multimodal classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultimodalClassifier(nn.Module):
    def __init__(self, audio_dim, text_dim, num_classes):
        super(MultimodalClassifier, self).__init__()

        self.audio_layer = nn.Linear(audio_dim, 128)
        self.text_layer = nn.Linear(text_dim, 128)

        self.fc_layer = nn.Linear(256, 128)  # 128 from audio + 128 from text
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, audio_features, text_features):
        audio_out = self.audio_layer(audio_features)
        text_out = self.text_layer(text_features)

        combined = torch.cat((audio_out, text_out), dim=1)

        out = self.fc_layer(combined)
        out = self.classifier(out)

        return out

# Dimensions based on your feature extractors
audio_dim = 768  # for example
text_dim = 512  # for example
num_classes = 7  # Number of emotion classes

model = MultimodalClassifier(audio_dim, text_dim, num_classes)

# Sample data
audio_features = torch.randn(32, audio_dim)  # Batch of 32 samples
text_features = torch.randn(32, text_dim)    # Batch of 32 samples
labels = torch.randint(0, num_classes, (32,))  # Random labels

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Forward pass
outputs = model(audio_features, text_features)

# Compute loss and update weights
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()
